In [5]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [7]:
pd.set_option('display.max_columns', None)

In [9]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [11]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6594

In [13]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:5]
        
    for pagina_url in lista_prueba:
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("article", class_="anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')   
    
        
    return lista_paginas, anuncios_coches

In [15]:
lista_paginas,anuncios_coches = links_coches()
anuncios_coches

['https://www.autocasion.com/coches-segunda-mano/fiat-500-ocasion/500-1-0-hybrid-dolcevita-52kw-ref14697758',
 'https://www.autocasion.com/coches-segunda-mano/bmw-serie-1-ocasion/118d-5-ref14636885',
 'https://www.autocasion.com/coches-segunda-mano/mercedes-benz-eqc-ocasion/eqc-400-4matic-ref14697751',
 'https://www.autocasion.com/coches-segunda-mano/bmw-x1-ocasion/x1-sdrive-16d-business-2-ref14697756',
 'https://www.autocasion.com/coches-segunda-mano/honda-civic-ocasion/civic-1-0-vtec-turbo-elegance-navi-1-ref14310581',
 'https://www.autocasion.com/coches-km0/km-0/audi-a1-km0/a1-sportback-25-tfsi-adrenalin-black-edition-ref14697752',
 'https://www.autocasion.com/coches-km0/km-0/honda-jazz-km0/jazz-1-5-i-mmd-advance-166eaafdad72ca-ref14347648',
 'https://www.autocasion.com/coches-segunda-mano/toyota-corolla-ocasion/1-8-125h-active-tech-e-cvt-166a0db4bd069d-ref13894390',
 'https://www.autocasion.com/coches-segunda-mano/ford-focus-ocasion/focus-1-0-ecoboost-mhev-92kw-st-line-125-5p-16-re

In [17]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [19]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [21]:
coches_df = dict()

for coche in tqdm.tqdm(anuncios_coches):
    print(coche)
    url_coche = coche
    response = session.get(url_coche, headers=headers)
    
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
        
    info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[-1].text
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
        for detalle in detalles_carroceria:
            detalle = detalle.text.replace('Medidas','').strip()
            
            if 'Largo' in detalle: 
                info_coche['largo'] = detalle
            elif 'Ancho' in detalle: 
                info_coche['ancho'] = detalle
            elif 'Alto' in detalle: 
                info_coche['alto'] = detalle
            #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
            elif 'Capacidad del maletero' in detalle: 
                info_coche['capacidad_maletero'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            elif 'Número de plazas' in detalle: 
                info_coche['num_plazas'] = detalle
            elif 'Número de puertas' in detalle: 
                info_coche['num_puertas'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            elif 'Batalla' in detalle: 
                info_coche['batalla'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            #supongo que si hay dos pesos el segundo será la masa max. autorizada
            elif 'Peso' in detalle: 
                info_coche['peso'] = detalle
           
        detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
        for detalle in detalles_consumos:
            detalle = ' '.join(detalle.text.replace('\n','').split())
            
            if 'Consumo medio' in detalle: 
                info_coche['consumo_medio'] = detalle
            elif 'Consumo en carretera' in detalle: 
                info_coche['consumo_carretera'] = detalle
            elif 'Consumo urbano' in detalle: 
                info_coche['consumo_urbano'] = detalle
            elif 'CO2' in detalle: 
                info_coche['CO2'] = detalle
            elif 'Depósito' in detalle: 
                info_coche['deposito'] = detalle

        detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
        for detalle in detalles_motor:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Combustible' in detalle: 
                info_coche['combustible'] = detalle
            elif 'Nº de cilindros' in detalle: 
                info_coche['cilindros'] = detalle
            elif 'Cilindrada' in detalle: 
                info_coche['cilindrada'] = detalle
            elif 'Sobrealimentación' in detalle: 
                info_coche['sobrealimentacion'] = detalle

        detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
        for detalle in detalles_transmision:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Tipo de transmisión' in detalle: 
                info_coche['tipo_cambio'] = detalle
            elif 'Tracción' in detalle: 
                info_coche['traccion'] = detalle
            elif 'Número de marchas' in detalle: 
                info_coche['num_marchas'] = detalle

        detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
        for detalle in detalles_prestaciones:
            detalle = detalle.text
            
            if 'Potencia (KW)' in detalle: 
                info_coche['potencia_kw'] = detalle
            elif 'Potencia (CV)' in detalle: 
                info_coche['potencia_cv'] = detalle
            elif 'Par' in detalle: 
                info_coche['par'] = detalle
            elif 'Velocidad' in detalle: 
                info_coche['velocidad_max'] = detalle
            elif 'Aceleración' in detalle: 
                info_coche['aceleracion'] = detalle
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(1, 3))


  0%|          | 0/80 [00:00<?, ?it/s]

https://www.autocasion.com/coches-segunda-mano/fiat-500-ocasion/500-1-0-hybrid-dolcevita-52kw-ref14697758


  1%|▏         | 1/80 [00:03<05:10,  3.93s/it]

https://www.autocasion.com/coches-segunda-mano/bmw-serie-1-ocasion/118d-5-ref14636885


  2%|▎         | 2/80 [00:07<04:32,  3.50s/it]

...

 98%|█████████▊| 78/80 [03:15<00:04,  2.06s/it]

https://www.autocasion.com/coches-segunda-mano/mercedes-benz-clase-glc-ocasion/glc-mercedes-benz-mercedesamg-glc-43-4matic-16703f1c9c4860-ref14512764


 99%|█████████▉| 79/80 [03:17<00:02,  2.31s/it]

https://www.autocasion.com/coches-segunda-mano/citroen-c3-aircross-ocasion/c3-aircross-puretech-s-s-plus-110-ref14382157


100%|██████████| 80/80 [03:20<00:00,  2.50s/it]


In [23]:
df = pd.DataFrame.from_dict(coches_df).T

df.iloc[30]

provincia                                                      Madrid
marca_modelo              Detalles LAND-ROVER Discovery Sport (2021) 
fecha_matriculacion                              Matriculado: 06/2021
kilometraje                                                     61740
nombre_vendedor                MOVILCAR, concesionario oficial Jaguar
tipo_vendedor                                             Profesional
distintivo_ambiental                                                C
garantia                                                     24 meses
precio_contado                                         31900 32490 2%
precio_nuevo                                                    50100
largo                                                   4597 mm Largo
ancho                                                   2069 mm Ancho
alto                                                     1727 mm Alto
capacidad_maletero                Capacidad del maletero (l)780/1.574
carrocería          

In [25]:
df

,provincia,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carrocería,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,carroceria
0,Málaga,Detalles FIAT 500 (2022),Matriculado: 06/2022,25000,AUTOGEST MALAGA,Profesional,ECO,12 meses,13995,19300,3571 mm Largo,1627 mm Ancho,1488 mm Alto,Capacidad del maletero (l)185/0,CarroceríaBerlina,Número de plazas4,Batalla (mm)\n2.300,Peso/Masa max. autorizado (kg)1.055,Número de puertas3,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 35,00 litros",Combustible Gasolina,Nº de cilindros 3 cilindros,Cilindrada (cm3) 999,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)52,Potencia (CV)70,Par (Nm)92,Velocidad máxima (km/h)167,"Aceleración 0-100 km/h (s)13,80",NaN
1,Madrid,Detalles BMW Serie 1 (2016),Matriculado: 10/2016,191824,GANGACARS,Profesional,C,12 meses,14990,29550,4329 mm Largo,1765 mm Ancho,1421 mm Alto,Capacidad del maletero (l)360/1.200,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.690,Peso/Masa max. autorizado (kg)1.915,Número de puertas5,"Consumo medio 4,00 litros/100 km","Consumo en carretera 3,50 litros/100 km","Consumo urbano 4,80 litros/100 km",CO2 (g/km) 104 g/km,"Depósito de combustible (l) 52,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.995,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Manual,TracciónTracción trasera,Número de marchas6,Potencia (KW)110,Potencia (CV)150,Par (Nm)320,Velocidad máxima (km/h)212,"Aceleración 0-100 km/h (s)8,30",NaN
2,Ubicación:,Detalles MERCEDES-BENZ EQC (2022),Matriculado: 01/2022,28900,GT CARS PREMIUM LAS PALMAS,Profesional,0 EMISIONES,12 meses,54900,85429,4761 mm Largo,1884 mm Ancho,1623 mm Alto,Capacidad del maletero (l)500/0,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\n2.873,Peso/Masa max. autorizado (kg)2.940,Número de puertas5,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 0,00 litros",Combustible Corriente eléctrica,Nº de cilindros - cilindros,Cilindrada (cm3) 0,Sobrealimentación -,"Tipo de transmisión Directo, sin caja de cambios",TracciónTodo terreno tracción total permanente,Número de marchas1,Potencia (KW)300,Potencia (CV)408,Par (Nm)760,Velocidad máxima (km/h)180,"Aceleración 0-100 km/h (s)5,10",NaN
3,Málaga,Detalles BMW X1 (2019),Matriculado: 10/2019,32000,AUTOGEST MALAGA,Profesional,C,12 meses,23900,36479,4447 mm Largo,1821 mm Ancho,1598 mm Alto,Capacidad del maletero (l)505/1.550,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\n2.670,Peso/Masa max. autorizado (kg)2.040,Número de puertas5,"Consumo medio 4,10 litros/100 km","Consumo en carretera 3,70 litros/100 km","Consumo urbano 4,70 litros/100 km",CO2 (g/km) 107 g/km,"Depósito de combustible (l) 51,00 litros",Combustible Diesel,Nº de cilindros 3 cilindros,Cilindrada (cm3) 1.496,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)85,Potencia (CV)116,Par (Nm)270,Velocidad máxima (km/h)190,"Aceleración 0-100 km/h (s)11,50",NaN
4,Valencia,Detalles HONDA Civic (2021),Matriculado: 08/2021,44818,HONDA QUADIS IBERTECNO VALENCIA,Profesional,C,24 meses,26490,27644,4518 mm Largo,1799 mm Ancho,1434 mm Alto,Capacidad del maletero (l)478/1.267,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.697,Peso/Masa max. autorizado (kg)1.775,Número de puertas5,"Consumo medio 4,80 litros/100 km","Consumo en carretera 4,10 litros/100 km","Consumo urbano 6,10 litros/100 km",

In [29]:
df.to_csv('../bin/data.csv', index=False)

## Pruebas fallos

In [31]:
url = 'https://www.autocasion.com/coches-segunda-mano/mercedes-benz-eqc-ocasion/eqc-400-4matic-ref14697751'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [39]:
 soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[-1].text

'Ubicación:'